In [2]:
import os

In [3]:
cdl_dir = f"C:\\Users\\admin\\Desktop\\desktop\\web\\scrapper\\cdl_verilog_data\\cdl\\cdl"
verilog_path = f"C:\\Users\\admin\\Desktop\\desktop\\web\\scrapper\\cdl_verilog_data\\im301E_digital.v"

In [41]:
with open(verilog_path, 'r') as f:
    verilog_data = f.readlines()

real_time_created_cells = {

}

out_file = 'out.txt'

out_lines = []

line_no = 0
while line_no < len(verilog_data):
    line = verilog_data[line_no].strip()
    if line.startswith('module'):
        cell_name = line.split()[1]
        out_line = f".SUBCKT {cell_name} "
        extract_pins = []
        # while not line.endswith(');'):
        #     if not line.startswith('module'):
        #         tmp_pin_name = line.replace(',', '').replace(';', '').replace(')', '')
        #         if tmp_pin_name != 'VDD' or tmp_pin_name != 'GND':
        #             extract_pins.append(
        #                 tmp_pin_name
        #             )
        #     line_no += 1
        #     line = verilog_data[line_no].strip()
        # tmp_pin_name = line.replace(',', '').replace(';', '').replace(')', '')
        # if tmp_pin_name != 'VDD' or tmp_pin_name != 'GND':
        #     extract_pins.append(
        #         tmp_pin_name
        #     )

        while not line.startswith('//'):
            if line.startswith('input') or line.startswith('output'):
                pin_name_should_be = line.strip().split()[1]

                if pin_name_should_be.startswith('['):
                    pin_name = line.strip().split()[-1].replace(';', '')
                    pin_array_max = int(pin_name_should_be.replace('[', '').split(':')[0])
                    pin_array_min = int(pin_name_should_be.replace(']', '').split(':')[1])
                    for i in range(pin_array_max, pin_array_min-1, -1):
                        out_line = out_line + f"{pin_name}[{i}] "
                        if pin_name not in extract_pins:
                            extract_pins.append(pin_name)
                        # extract_pins.append(f"{pin_name}[{i}]")
                else:
                    pin_name = pin_name_should_be.replace(';', '')
                    out_line = out_line + f"{pin_name} "
                    if pin_name not in extract_pins:
                        extract_pins.append(pin_name)
            line_no += 1
            if line_no >= len(verilog_data):
                break
            line = verilog_data[line_no].strip()

        out_lines.append(out_line)
        out_lines.append("")
        real_time_created_cells[cell_name] = extract_pins
        # print(real_time_created_cells.keys())

        while not line.startswith('endmodule'):
            if line.endswith('(') and not line.startswith('module'):
                cdl_cell = line.split()[0]
                net_name = line.split()[1]
                try:
                    cdl_file = os.path.join(cdl_dir, f"{cdl_cell}.cdl")
                    with open(cdl_file, 'r') as fc:
                        cdl_data = fc.readlines()

                    for line_c in cdl_data:
                        if line_c.startswith('.SUBCKT'):
                            pin_seq = line_c.split()[2:]
                            break
                except:
                    try:
                        pin_seq = real_time_created_cells[cdl_cell]
                    except:
                        pin_seq = []
                    # print(cdl_cell)

                temp_line_no = line_no + 1
                temp_line = verilog_data[temp_line_no].strip()

                net_dict = {}
                while temp_line.startswith('.'):
                    if temp_line.strip().startswith('.'):
                        out_net = temp_line.split('(')[0].replace('.', '')
                        in_net = temp_line.split('(')[-1].replace('),', '')
                        net_dict[out_net] = in_net

                    temp_line_no = temp_line_no + 1
                    temp_line = verilog_data[temp_line_no].strip()
                    if temp_line.startswith('endmodule'):
                        break
                # print(net_dict, pin_seq)

                additional_line = f'X{net_name} '
                for pn in pin_seq:
                    if pn in net_dict.keys():
                        additional_line = additional_line + f"{net_dict[pn]} "
                out_lines.append(additional_line)

            line_no += 1
            if line_no >= len(verilog_data):
                break
            line = verilog_data[line_no].strip()
        out_lines.append("")
        out_lines.append(".ENDS")
        out_lines.append("")
        out_lines.append("")
    line_no += 1
    if line_no >= len(verilog_data):
        break
    line = verilog_data[line_no].strip()


with open(out_file, 'w') as f:
    f.write("\n".join(out_lines))
